In [ ]:
import backtrader as bt
import backtrader.feeds as btfeeds
import backtrader.indicators as btind
import itertools
from matplotlib.dates import date2num
import pandas as pd
import datetime


In [ ]:

class OHLCAVWithDisaggCoT(btfeeds.GenericCSVData):
  
    lines = ('Tot_Rept_Positions_Long_All',
            'M_Money_Positions_Long_All',
            'Tot_Rept_Positions_Short_All',
            'M_Money_Positions_Short_All',)
    

    params = (
        ('nullvalue', 0.0),
        ('dtformat', ('%Y-%m-%d')),

        ('Date', None),
        ('Open', 1),
        ('High', 2),
        ('Low', 3),
        ('Close', 4),
        ('Adj Close', 5),
        ('Volume', 6),
        ('Tot_Rept_Positions_Long_All',7),
        ('Tot_Rept_Positions_Short_All',8),
        ('M_Money_Positions_Long_All',9),
        ('M_Money_Positions_Short_All',10),
    )
    
class PandasData(btfeeds.DataBase):
    '''
    The ``dataname`` parameter inherited from ``feed.DataBase`` is the pandas
    DataFrame
    '''

    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('datetime', -1),

        # Possible values below:
        #  None : column not present
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', -1),
        ('volume', -1),
        ('Tot_Rept_Positions_Long_All',-1),
        ('Tot_Rept_Positions_Short_All',-1),
        ('M_Money_Positions_Long_All',-1),
        ('M_Money_Positions_Short_All',-1),
    )

In [ ]:
class MyStrategy(bt.Strategy):
    params = (('ind', 'sma'), ('period', 15),)
    INDS = ['bollinger']
    def __init__(self):
        bt.talib.BBANDS(self.data, timeperiod=25,
                            plotname='TA_BBANDS')
        bt.indicators.BollingerBands(self.data, period=25)
    def next(self):
        if self.sma > self.data.close:
            # Do something
            pass

        elif self.sma < self.data.close:
            # Do something else
            pass

In [ ]:
import backtrader.strategies.sma_crossover as sma

class MySignal(bt.Indicator):
    lines = ('sell_signal', 'buy_signal', 'break_up',
             'break_down', 'Outstanding_Positions_Tot', 'Outstanding_Positions_M_Money')
    params = (('Lookback_for_outstanding', 50), ('Longest_lookback_for_breakning', 50))

    def __init__(self):
        print('init')
        self.lines.signal = self.data - bt.indicators.SMA(period=self.p.period)
    def next():
        pass

In [ ]:
dataname='../data/GC=F_com_disagg.csv'
# data = OHLCAVWithDisaggCoT(dataname=dataname)

cerebro = bt.Cerebro()
df = pd.read_csv(dataname, index_col=0)
df.index = pd.to_datetime(df.index)
# df


In [ ]:
data = bt.feeds.PandasData(dataname=df)
# data = OHLCAVWithDisaggCoT(dataname = dataname)
cerebro.adddata(data)

# cerebro.add_signal(bt.SIGNAL_LONGSHORT, MySignal)

# 初始资金 100,000,000
cerebro.broker.setcash(1e8)
# 佣金，双边各 0.0003
cerebro.broker.setcommission(commission=0.0003)
# 滑点：双边各 0.0001
cerebro.broker.set_slippage_perc(perc=0.0001)

cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='pnl') # 返回收益率时序数据
cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='_AnnualReturn') # 年化收益率
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='_SharpeRatio') # 夏普比率
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='_DrawDown') # 回撤

# 启动回测
result = cerebro.run()
# 从返回的 result 中提取回测结果
strat = result[0]
# 返回日度收益率序列
daily_return = pd.Series(strat.analyzers.pnl.get_analysis())
# 打印评价指标
print("--------------- AnnualReturn -----------------")
print(strat.analyzers._AnnualReturn.get_analysis())
print("--------------- SharpeRatio -----------------")
print(strat.analyzers._SharpeRatio.get_analysis())
print("--------------- DrawDown -----------------")
print(strat.analyzers._DrawDown.get_analysis())